# Generating non-obviously robotic music

In [1]:
import math
import os
import time

import magenta.music as mm
import tensorflow as tf
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.music import DEFAULT_QUARTERS_PER_MINUTE
from magenta.protobuf.generator_pb2 import GeneratorOptions
from magenta.protobuf.music_pb2 import NoteSequence
from visual_midi import Plotter

In [2]:
from magenta.models.shared import sequence_generator_bundle 

In [3]:
from magenta.models.performance_rnn import performance_sequence_generator

In [4]:
bundle_name = "performance_with_dynamics.mag"

In [5]:
mm.notebook_utils.download_bundle(bundle_name, "bundles")

In [6]:
bundle = sequence_generator_bundle.read_bundle_file(
    os.path.join("bundles", bundle_name))

In [7]:
generator_map = performance_sequence_generator.get_generator_map()

In [8]:
for key, value in generator_map.items():
    print (key)

performance
performance_with_dynamics
performance_with_dynamics_compact
performance_with_dynamics_and_modulo_encoding
performance_with_dynamics_and_note_encoding
density_conditioned_performance_with_dynamics
pitch_conditioned_performance_with_dynamics
multiconditioned_performance_with_dynamics
optional_multiconditioned_performance_with_dynamics


**performance** supports expressive timing, and **performance_with_dynamics** supports different note strike velocities

These are the types of performance RNN, 

In [9]:
generator = generator_map['performance_with_dynamics'](checkpoint=None, bundle=bundle)

Jupyter notebooks often give weird bugs, I had one with the line above, saying that the bundle name didn't match the type of model it was, but it all worked fine on restarting the notebook. Annoyingly, notebooks keep variables alive in their internal state even if the code is gone, so sometimes restarting the whole notebook is the only way. 

In [12]:
generator.initialize()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\alecr\AppData\Local\Temp\tmpy4axooxl\model.ckpt


In [13]:
primer_sequence = NoteSequence()

In [14]:
qpm = DEFAULT_QUARTERS_PER_MINUTE

In [15]:
seconds_per_step = 60.0 / qpm / getattr(generator, "steps_per_quarter", 4)

In [16]:
primer_sequence_length_steps = math.ceil(primer_sequence.total_time
                                           / seconds_per_step)
primer_sequence_length_time = primer_sequence_length_steps * seconds_per_step

In [17]:
primer_end_adjust = (0.00001 if primer_sequence_length_time > 0 else 0)
primer_start_time = 0
primer_end_time = (primer_start_time
                     + primer_sequence_length_time
                     - primer_end_adjust)

In [18]:
total_length_steps = 512

In [19]:
generation_length_steps = total_length_steps - primer_sequence_length_steps

In [20]:
if generation_length_steps <= 0:
    raise Exception("Total length in steps too small "
                    + "(" + str(total_length_steps) + ")"
                    + ", needs to be at least one bar bigger than primer "
                    + "(" + str(primer_sequence_length_steps) + ")")
generation_length_time = generation_length_steps * seconds_per_step

In [21]:
generation_start_time = primer_end_time
generation_end_time = (generation_start_time
                         + generation_length_time
                         + primer_end_adjust)

In [22]:
print(f"Primer time: [{primer_start_time}, {primer_end_time}]")
print(f"Generation time: [{generation_start_time}, {generation_end_time}]")

Primer time: [0, 0.0]
Generation time: [0.0, 64.0]


In [23]:
generator_options = GeneratorOptions()

In [26]:
generator_options.args['temperature'].float_value = 1
generator_options.args['beam_size'].int_value = 2
generator_options.args['branch_factor'].int_value = 2
generator_options.args['steps_per_iteration'].int_value = 1

In [28]:
notes_per_second = None

In [29]:
if notes_per_second:
    generator_options.args['notes_per_second'].string_value = notes_per_second

In [30]:
pitch_class_histogram = None
if pitch_class_histogram:
    generator_options.args['pitch_class_histogram'].string_value = (
    pitch_class_histogram)

In [31]:
generator_options.generate_sections.add(
    start_time=generation_start_time,
    end_time=generation_end_time)

end_time: 64.0

In [32]:
sequence = generator.generate(primer_sequence, generator_options)

INFO:tensorflow:Need to generate 6299 more steps for this sequence, will try asking for 3780 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -2902.812744 


In [33]:
type(sequence)

music_pb2.NoteSequence

In [36]:
mm.midi_io.note_sequence_to_midi_file(sequence, 'performance1.mid')